In [27]:
import pandas as pd
import numpy as np
from glob import glob, iglob
import pymongo
import json

In [2]:
filenames= glob('data/Current_BioDiv/*.xlsx')
filenames

['data/Current_BioDiv\\NPSpecies_FullList_DENA_20210310195014.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_GAAR_20210310195036.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_GLBA_20210310195110.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_KATM_20210310195135.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_KEFJ_20210310195156.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_KOVA_20210310195246.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_LACL_20210310195303.xlsx',
 'data/Current_BioDiv\\NPSpecies_FullList_WRST_20210310195403.xlsx']

In [3]:
all_parks_df=pd.DataFrame()

for f in filenames:
    data = pd.read_excel(f, 'Sheet1')
    all_parks_df = all_parks_df.append(data)

all_parks_df

,Park Code,Park Name,Category,Category Sort,Order,Family,Taxon Code,TSN,Taxon Record Status,Scientific Name,Common Names,Synonyms,Record Status,Occurrence,Occurrence Tags,Nativeness,Nativeness Tags,Abundance
0,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95055,180706.0,Active,Bison bison,Bison,NaN,Approved,Not In Park,Historical,Native,NaN,NaN
1,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95063,180713.0,Active,Oreamnos americanus,Mountain goat,NaN,Approved,Not In Park,Adjacent,Native,NaN,NaN
2,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95060,180710.0,Active,Ovis dalli,Dall's sheep,NaN,Approved,Present,NaN,Native,NaN,Unknown
3,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Cervidae,95052,180703.0,Active,Alces alces,Moose,NaN,Approved,Present,NaN,Native,NaN,Common
4,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Cervidae,95050,180701.0,Active,Rangifer tarandus,Caribou,Rangifer arcticus,Approved,Present,NaN,Native,NaN,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,WRST,Wrangell - St Elias National Park and Preserve,Insect,11,Lepidoptera,Tortricidae,12472,117856.0,Active,Tortricidae,"leaf rollers, leaf tyers",NaN,Approved,Present,NaN,Native,NaN,Unknown
1792,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,NaN,Enchytraeidae,985968,980415.0,Active,Mesenchytraeus solifugus,Ice worm,NaN,Approved,Present,NaN,Native,NaN,Unknown
1793,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,Enterogona,Ascidiidae,68721,159176.0,Active,Ascidia callosa,NaN,NaN,Approved,Probably Present,NaN,Native,NaN,NaN
1794,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,Pleurogona,Styelidae,926824,NaN,Active,Botrylloides violaceus,Golden Chain tunicate,NaN,Approved,Not In Park,Adjacent,Non-native,NaN,NaN


In [4]:
all_parks_df.reset_index(inplace=True)

In [5]:
park_loc='data/Alaska_parks.csv'
parks_loc_df = pd.read_csv(park_loc)
parks_loc_df

,Unnamed: 0,park_code,national_park,latitude,longitude,nearest city
0,14,DENA,Denali National Park,63.33,-150.50,Wasilla
1,18,GAAR,Gates of the Arctic National Park,67.78,-153.30,College
2,20,GLBA,Glacier Bay National Park,58.50,-137.00,Sitka
3,32,KATM,Katmai National Park,58.50,-155.00,Saint Paul Harbor
4,33,KEFJ,Kenai Fjords National Park,59.92,-149.65,Sterling
5,34,KOVA,Kobuk Valley National Park,67.55,-159.28,Barrow
6,35,LACL,Lake Clark National Park,60.97,-153.42,Homer
7,52,WRST,Wrangell - St Elias National Park,61.00,-142.00,Haines Junction


In [6]:
parks_loc_df=parks_loc_df.rename(columns={"park_code":"Park Code","nearest city": "City"})
parks_loc_df

,Unnamed: 0,Park Code,national_park,latitude,longitude,City
0,14,DENA,Denali National Park,63.33,-150.50,Wasilla
1,18,GAAR,Gates of the Arctic National Park,67.78,-153.30,College
2,20,GLBA,Glacier Bay National Park,58.50,-137.00,Sitka
3,32,KATM,Katmai National Park,58.50,-155.00,Saint Paul Harbor
4,33,KEFJ,Kenai Fjords National Park,59.92,-149.65,Sterling
5,34,KOVA,Kobuk Valley National Park,67.55,-159.28,Barrow
6,35,LACL,Lake Clark National Park,60.97,-153.42,Homer
7,52,WRST,Wrangell - St Elias National Park,61.00,-142.00,Haines Junction


In [7]:
merged1_df = all_parks_df.merge(parks_loc_df, on='Park Code')
merged1_df

,index,Park Code,Park Name,Category,Category Sort,Order,Family,Taxon Code,TSN,Taxon Record Status,...,Occurrence,Occurrence Tags,Nativeness,Nativeness Tags,Abundance,Unnamed: 0,national_park,latitude,longitude,City
0,0,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95055,180706.0,Active,...,Not In Park,Historical,Native,NaN,NaN,14,Denali National Park,63.33,-150.5,Wasilla
1,1,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95063,180713.0,Active,...,Not In Park,Adjacent,Native,NaN,NaN,14,Denali National Park,63.33,-150.5,Wasilla
2,2,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95060,180710.0,Active,...,Present,NaN,Native,NaN,Unknown,14,Denali National Park,63.33,-150.5,Wasilla
3,3,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Cervidae,95052,180703.0,Active,...,Present,NaN,Native,NaN,Common,14,Denali National Park,63.33,-150.5,Wasilla
4,4,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Cervidae,95050,180701.0,Active,...,Present,NaN,Native,NaN,Unknown,14,Denali National Park,63.33,-150.5,Wasilla
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13431,1791,WRST,Wrangell - St Elias National Park and Preserve,Insect,11,Lepidoptera,Tortricidae,12472,117856.0,Active,...,Present,NaN,Native,NaN,Unknown,52,Wrangell - St Elias National Park,61.00,-142.0,Haines Junction
13432,1792,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,NaN,Enchytraeidae,985968,980415.0,Active,...,Present,NaN,Native,NaN,Unknown,52,Wrangell - St Elias National Park,61.00,-142.0,Haines Junction
13433,1793,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,Enterogona,Ascidiidae,68721,159176.0,Active,...,Probably Present,NaN,Native,NaN,NaN,52,Wrangell - St Elias National Park,61.00,-142.0,Haines Junction
13434,1794,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,Pleurogona,Styelidae,926824,NaN,Active,...,Not In Park,Adjacent,Non-native,NaN,NaN,52,Wrangell - St Elias National Park,61.00,-142.0,Haines Junction


In [8]:
weather='data/city_weather.csv'
weather_df = pd.read_csv(weather)
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed
0,0,Wasilla,61.58,-149.44,12.20,10.99,52,1,3.44
1,1,College,64.86,-147.80,-5.01,-18.40,84,1,2.80
2,2,Sitka,57.05,-135.33,33.80,33.80,55,40,14.97
3,3,Saint Paul Harbor,57.79,-152.41,30.20,30.20,37,1,17.27
4,4,Sterling,39.01,-77.43,57.99,55.00,18,75,5.91
5,5,Barrow,71.29,-156.79,-9.40,-9.40,70,1,16.11
6,6,Homer,59.64,-151.55,17.60,14.00,49,1,8.05
7,7,Haines Junction,60.75,-137.51,3.20,3.20,47,83,14.97


In [9]:
merged2_df = merged1_df.merge(weather_df, on='City')
merged2_df

,index,Park Code,Park Name,Category,Category Sort,Order,Family,Taxon Code,TSN,Taxon Record Status,...,longitude,City,Unnamed: 0_y,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed
0,0,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95055,180706.0,Active,...,-150.5,Wasilla,0,61.58,-149.44,12.2,10.99,52,1,3.44
1,1,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95063,180713.0,Active,...,-150.5,Wasilla,0,61.58,-149.44,12.2,10.99,52,1,3.44
2,2,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Bovidae,95060,180710.0,Active,...,-150.5,Wasilla,0,61.58,-149.44,12.2,10.99,52,1,3.44
3,3,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Cervidae,95052,180703.0,Active,...,-150.5,Wasilla,0,61.58,-149.44,12.2,10.99,52,1,3.44
4,4,DENA,Denali National Park and Preserve,Mammal,1,Artiodactyla,Cervidae,95050,180701.0,Active,...,-150.5,Wasilla,0,61.58,-149.44,12.2,10.99,52,1,3.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13431,1791,WRST,Wrangell - St Elias National Park and Preserve,Insect,11,Lepidoptera,Tortricidae,12472,117856.0,Active,...,-142.0,Haines Junction,7,60.75,-137.51,3.2,3.20,47,83,14.97
13432,1792,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,NaN,Enchytraeidae,985968,980415.0,Active,...,-142.0,Haines Junction,7,60.75,-137.51,3.2,3.20,47,83,14.97
13433,1793,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,Enterogona,Ascidiidae,68721,159176.0,Active,...,-142.0,Haines Junction,7,60.75,-137.51,3.2,3.20,47,83,14.97
13434,1794,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,12,Pleurogona,Styelidae,926824,NaN,Active,...,-142.0,Haines Junction,7,60.75,-137.51,3.2,3.20,47,83,14.97


In [10]:
# merged2_df = all_parks_df.merge(parks_loc_df, on='Park Code')

clean_merge=merged2_df.drop(columns=["Taxon Code","TSN","Category Sort","Unnamed: 0_y","national_park","Nativeness Tags"\
                                     ,"Synonyms","Occurrence Tags"])
clean_merge


,index,Park Code,Park Name,Category,Order,Family,Taxon Record Status,Scientific Name,Common Names,Record Status,...,latitude,longitude,City,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed
0,0,DENA,Denali National Park and Preserve,Mammal,Artiodactyla,Bovidae,Active,Bison bison,Bison,Approved,...,63.33,-150.5,Wasilla,61.58,-149.44,12.2,10.99,52,1,3.44
1,1,DENA,Denali National Park and Preserve,Mammal,Artiodactyla,Bovidae,Active,Oreamnos americanus,Mountain goat,Approved,...,63.33,-150.5,Wasilla,61.58,-149.44,12.2,10.99,52,1,3.44
2,2,DENA,Denali National Park and Preserve,Mammal,Artiodactyla,Bovidae,Active,Ovis dalli,Dall's sheep,Approved,...,63.33,-150.5,Wasilla,61.58,-149.44,12.2,10.99,52,1,3.44
3,3,DENA,Denali National Park and Preserve,Mammal,Artiodactyla,Cervidae,Active,Alces alces,Moose,Approved,...,63.33,-150.5,Wasilla,61.58,-149.44,12.2,10.99,52,1,3.44
4,4,DENA,Denali National Park and Preserve,Mammal,Artiodactyla,Cervidae,Active,Rangifer tarandus,Caribou,Approved,...,63.33,-150.5,Wasilla,61.58,-149.44,12.2,10.99,52,1,3.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13431,1791,WRST,Wrangell - St Elias National Park and Preserve,Insect,Lepidoptera,Tortricidae,Active,Tortricidae,"leaf rollers, leaf tyers",Approved,...,61.00,-142.0,Haines Junction,60.75,-137.51,3.2,3.20,47,83,14.97
13432,1792,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,NaN,Enchytraeidae,Active,Mesenchytraeus solifugus,Ice worm,Approved,...,61.00,-142.0,Haines Junction,60.75,-137.51,3.2,3.20,47,83,14.97
13433,1793,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,Enterogona,Ascidiidae,Active,Ascidia callosa,NaN,Approved,...,61.00,-142.0,Haines Junction,60.75,-137.51,3.2,3.20,47,83,14.97
13434,1794,WRST,Wrangell - St Elias National Park and Preserve,Other Non-vertebrates,Pleurogona,Styelidae,Active,Botrylloides violaceus,Golden Chain tunicate,Approved,...,61.00,-142.0,Haines Junction,60.75,-137.51,3.2,3.20,47,83,14.97


In [11]:
clean_merge["Park Code"].unique()

array(['DENA', 'GAAR', 'GLBA', 'KATM', 'KEFJ', 'KOVA', 'LACL', 'WRST'],
      dtype=object)

In [12]:
clean_merge["Category"].unique()

array(['Mammal', 'Bird', 'Amphibian', 'Fish', 'Vascular Plant',
       'Non-vascular Plant', 'Insect', 'Other Non-vertebrates', 'Fungi',
       'Reptile', 'Crab/Lobster/Shrimp', 'Slug/Snail', 'Spider/Scorpion',
       'Chromista', 'Bacteria'], dtype=object)

In [13]:
#duplicates park code, but rest runs as correctly

def myfunction(df):
    animal=df["Category"].unique()
    parks=df["Park Code"].unique()
    cities=df["City"].unique()
    p_name=df["Park Name"].unique()
    max_t=df["Max Temp"].unique()
        
    my_dictionary={}
    for each_park in parks:
        park_dict={}
        park_df=df[df['Park Code']==each_park].reset_index()
        
        for park_name in p_name:
            dict_list2=[]
            for x in range(len(park_df)):
                if park_df["Park Name"][x]==park_name:
                    dict_2= { 
                           'park name':park_df['Park Name'][x], 
                           'city':park_df['City'][x],
                           'max_temp':park_df['Max Temp'][x],
                           'min_temp':park_df['Min Temp'][x],
                           'wind_speed':park_df['Wind Speed'][x] 
                    }
                    dict_list2.append(dict_2)
                    dict_list3=list()
                    for sublist in dict_list2:
                        if sublist not in dict_list3:
                            dict_list3.append(sublist)
                    park_dict[each_park]=dict_list3
            
                
        for kind in animal:
            dict_list =[]
            for i in range(len(park_df)):
                if park_df['Category'][i]== kind: 
#                        print(park_df['Category'][i], '|', kind)
                        dict_1 = {
                           'name': park_df['Common Names'][i],
                           'occurance': park_df['Occurrence'][i],
                           'abundance': park_df['Abundance'][i]
                           }
                        dict_list.append(dict_1)
                park_dict[kind] = dict_list
        my_dictionary[each_park]=park_dict
    return my_dictionary
results=myfunction(clean_merge)
results

{'DENA': {'DENA': [{'park name': 'Denali National Park and Preserve',
    'city': 'Wasilla',
    'max_temp': 12.2,
    'min_temp': 10.99,
    'wind_speed': 3.44}],
  'Mammal': [{'name': 'Bison', 'occurance': 'Not In Park', 'abundance': nan},
   {'name': 'Mountain goat', 'occurance': 'Not In Park', 'abundance': nan},
   {'name': "Dall's sheep", 'occurance': 'Present', 'abundance': 'Unknown'},
   {'name': 'Moose', 'occurance': 'Present', 'abundance': 'Common'},
   {'name': 'Caribou', 'occurance': 'Present', 'abundance': 'Unknown'},
   {'name': 'Coyote', 'occurance': 'Present', 'abundance': 'Occasional'},
   {'name': 'Gray wolf, Wolf', 'occurance': 'Present', 'abundance': 'Common'},
   {'name': 'Red fox', 'occurance': 'Present', 'abundance': 'Unknown'},
   {'name': 'Lynx', 'occurance': 'Present', 'abundance': 'Unknown'},
   {'name': 'Wolverine', 'occurance': 'Present', 'abundance': 'Unknown'},
   {'name': 'River otter', 'occurance': 'Probably Present', 'abundance': nan},
   {'name': 'Amer

In [15]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define the 'classDB' database in Mongo
db = client.alaska_parksDB

In [16]:
a_parks=db.parks

In [34]:
db.alaska_parksDB.parks.insert(results)

<ipython-input-34-583041fd8f27>:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  db.alaska_parksDB.parks.insert(results)


DuplicateKeyError: E11000 duplicate key error collection: alaska_parksDB.alaska_parksDB.parks index: _id_ dup key: { _id: ObjectId('605032e024208e5bfc3299cf') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('605032e024208e5bfc3299cf')}, 'errmsg': "E11000 duplicate key error collection: alaska_parksDB.alaska_parksDB.parks index: _id_ dup key: { _id: ObjectId('605032e024208e5bfc3299cf') }"}